# <h1 align="center"><font color="gree">Construindo um Analista Financeiro com MCP e CrewAI</font></h1>

<font color="pink">Senior Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

In [6]:
import re
import json
import os
import yfinance as yf
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import CodeInterpreterTool, FileReadTool
from pydantic import BaseModel, Field


class QueryAnalysisOutput(BaseModel):
    """Saída estruturada para a tarefa de análise de consulta."""
    symbol: str = Field(description="Símbolo de cotação de ações. Exemplo: TSLA, AAPL.")
    timeframe: str = Field(description="Período de tempo. Exemplo: '1 dia', '1 mês', '1 ano'.")
    action: str = Field(description="Ação a ser realizada. Exemplo: 'buscar', 'plotar'.")
    


In [7]:
llm = LLM(
    model="ollama/deepseek-r1:7b",
    base_url="http://localhost:11434",
    # temperature=0.7
)

# 1) Agente de análise de consulta:
query_parser_agent = Agent(
    role="Analista de Dados de Ações",
    goal="Extrair detalhes das ações e buscar dados necessários a partir da consulta do usuário: {query}.",
    backstory="Você é um analista financeiro especializado em recuperação de dados de mercado de ações.",
    llm=llm,
    verbose=True,
    memory=True,
)

query_parsing_task = Task(
    description="Analise a consulta do usuário e extraia detalhes das ações.",
    expected_output="Um dicionário com as chaves: 'symbol', 'timeframe', 'action'.",
    output_pydantic=QueryAnalysisOutput,
    agent=query_parser_agent,
)


# 2) Agente de escrita de código:
code_writer_agent = Agent(
    role="Desenvolvedor Python Senior",
    goal="Escrever código Python para visualizar dados de ações.",
    backstory="""Você é um desenvolvedor Python Senior especializado em visualização de dados de mercado de ações. 
                 Você também é um especialista em bibliotecas Pandas, Matplotlib e yfinance.
                 Você é habilidoso em escrever código Python pronto para produção.""",
    llm=llm,
    verbose=True,
)

code_writer_task = Task(
    description="""Escreva código Python para visualizar dados de ações com base nas entradas do analista de ações,
                   onde você encontrará o símbolo da ação, o período de tempo e a ação.""",
    expected_output="Um arquivo Python (.py) limpo e executável para visualização de ações.",
    agent=code_writer_agent,
)


# 3) Agente interpretador de código (usa a ferramenta de interpretação de código da Crewai):
code_interpreter_tool = CodeInterpreterTool()

code_execution_agent = Agent(
    role="Especialista Sênior em Execução de Código",
    goal="Revise e execute o código Python gerado pelo agente de escrita de código para visualizar dados de estoque.",
    backstory="Você é um especialista em execução de código. Você é habilidoso em executar código Python.",
    # tools=[code_interpreter_tool],
    allow_code_execution=True,   # Esta linha automaticamente adiciona o CodeInterpreterTool
    llm=llm,
    verbose=True,
)

code_execution_task = Task(
    description="""Revise e execute o código Python gerado pelo agente de escrita de código para visualizar dados de estoque.""",
    expected_output="Um arquivo de script Python limpo e executável (.py) para visualização de ações.",
    agent=code_execution_agent,
)


In [8]:
### --- Configuração CREW --- ###

crew = Crew(
    agents=[query_parser_agent, code_writer_agent, code_execution_agent],
    tasks=[query_parsing_task, code_writer_task, code_execution_task],
    process=Process.sequential
)

# Executa a crewAI com uma consulta de exemplo:
result = crew.kickoff(inputs={"query": "Plote a ação da Tesla (TSLA) no período de 2024 até 2025"})

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Analista de Dados de Ações                                                                              │
│                                                                                                                 │
│  Task: Analise a consulta do usuário e extraia detalhes das ações.                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Analista de Dados de Ações                                                                              │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  <think>                                                                                                        │
│  Primeiro, preciso identificar a ação da Tesla, que é TSLA. Abaixo está a sigla do ícone correspondente.        │
│                                                                                                                 │
│  Em seguida, determino o período para Plotar a ação: de 2024 até 2025.                                          │
│                                                                                                                 │
│  Por fim, formatai os dados在我的分析中使用，确保严格按照用户要求的JSON格式返回。                               │
│  </think>                                                                                                       │
│                                                                                                                 │
│  ```json                                                                                                        │
│  {                                                                                                              │
│    "symbol": "TSLA",                                                                                            │
│    "timeframe": "1D",                                                                                           │
│    "action": "Plotar o gráfico da ação TSLA no período de 2024 a 2025."                                         │
│  }                                                                                                              │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Desenvolvedor Python Senior                                                                             │
│                                                                                                                 │
│  Task: Escreva código Python para visualizar dados de ações com base nas entradas do analista de ações,         │
│                     onde você encontrará o símbolo da ação, o período de tempo e a ação.                        │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Desenvolvedor Python Senior                                                                             │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  <think>                                                                                                        │
│  Okay, eu preciso criar um código Python para visualizar dados de ações com base nas entradas do analista.      │
│  Primeiro, identifico a ação da Tesla, cujo ícone é TSLA. Em seguida, determino o período para Plotar a ação:   │
│  de 2024 até 2025.                                                                                              │
│                                                                                                                 │
│  Principais passos:                                                                                             │
│                                                                                                                 │
│  1. **Importar bibliotecas necessárias**:                                                                       │
│     - `pandas` para manipular dados.                                                                            │
│     - `yfinance` para obter dados de ações.                                                                     │
│     - `matplotlib.pyplot` para plotar gráficos.                                                                 │
│                                                                                                                 │
│  2. **Obter os dados da ação**:                                                                                 │
│     - Utilizo o método `get_data` do yfinance para extrair os dados da ação TSLA no período especificado (de    │
│  2024 a 2025).                                                                                                  │
│                                                                                                                 │
│  3. **Formatar os dados**:                                                                                      │
│     - Converto os resultados para um DataFrame do Pandas para facilitar a manipulação e visualização.           │
│                                                                                                                 │
│  4. **Plotar o gráfico**:                                                                                       │
│     - Utilizo o método `plot` do DataFrame para criar um gráfico simples.                                       │
│     - Personalizo as cores dos traços para melhor visualização.                                                 │
│                                                                                                                 │
│  5. **Alterar os estilos de plotagem** (opcional):                                                              │
│     - Configuro tamanho da figura, fonte dos eixos e grid para tornar o gráfico mais legível.                   │
│                                                                                                                 │
│  6. **Salvar e exibir o gráfico**:                                                                              │
│     - Salvo o gráfico em um arquivo .png para armazenamento.                                                    │
│     - Exibo o gráfico na interface de visualização.                                                             │
│                                                        

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Especialista Sênior em Execução de Código                                                               │
│                                                                                                                 │
│  Task: Revise e execute o código Python gerado pelo agente de escrita de código para visualizar dados de        │
│  estoque.                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Especialista Sênior em Execução de Código                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  <think>                                                                                                        │
│  Okay, eu preciso criar um código Python para visualizar os dados de ações de Tesla (TSLA) no período de 2024   │
│  a 2025. Primeiro, leio o código que o agente de escrita de código generateu e entendo como ele funciona.       │
│                                                                                                                 │
│  O código utiliza a biblioteca yfinance para obter os dados da ação TSLA. Ele downloada os dados diários no     │
│  período despejado e imprime o gráfico do preço de fechamento. No entanto, o código é incompleto e contém       │
│  algumas instruções confuses sobre como salvar o gráfico e personalizar as configurações.                       │
│                                                                                                                 │
│  Para executar o código corretamente, preciso garantir que tenha todas as bibliotecas instaladas: pandas,       │
│  yfinance e matplotlib. Depois, preciso ajustar os parâmetros para obter os dados no período correto e          │
│  personalizar o gráfico conforme necessário.                                                                    │
│                                                                                                                 │
│  Finalmente, salvarei o gráfico em um arquivo .png para que possa visualizá-lo separadamente.                   │
│  </think>                                                                                                       │
│                                                                                                                 │
│  ```python                                                                                                      │
│  import pandas as pd                                                                                            │
│  import yfinance as yf                                                                                          │
│  import matplotlib.pyplot as plt                                                                                │
│                                                                                                                 │
│  # Passo 1: Obter os dados da ação Tesla (TSLA) no período de 2024 a 2025                                       │
│  tsla_dados = yf.download('TSLA', start='2024-01-01', end='2025-12-31')                                         │
│                                                                                                                 │
│  # Passo 2: Formatar os dados para DataFrame do Pandas                                                          │
│  df_tsla = pd.DataFrame(tsla_dados)                                                                             │
│                                                                                                                 │
│  # Passo 3: Plotar o gráfico da ação                                                                            │
│  plt.figure(figsize=(10, 6))                                                                                    │
│  plt.plot(df_tsla['Close'], label='Preço de Fechamento', color='red')                                           │
│  plt.title('Gráfico da Ação Tesla (TSLA) - 2024 a 2025'

In [9]:
print(result.raw)

<think>
Okay, eu preciso criar um código Python para visualizar os dados de ações de Tesla (TSLA) no período de 2024 a 2025. Primeiro, leio o código que o agente de escrita de código generateu e entendo como ele funciona.

O código utiliza a biblioteca yfinance para obter os dados da ação TSLA. Ele downloada os dados diários no período despejado e imprime o gráfico do preço de fechamento. No entanto, o código é incompleto e contém algumas instruções confuses sobre como salvar o gráfico e personalizar as configurações.

Para executar o código corretamente, preciso garantir que tenha todas as bibliotecas instaladas: pandas, yfinance e matplotlib. Depois, preciso ajustar os parâmetros para obter os dados no período correto e personalizar o gráfico conforme necessário.

Finalmente, salvarei o gráfico em um arquivo .png para que possa visualizá-lo separadamente.
</think>

```python
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

# Passo 1: Obter os dados da ação T